# Basic Data Cleaning and Manipulation #

### Task ###
Clean the raw data to handle missing values, outliers, or inconsistencies.

### Requirements ###

Write a script/notebook (scripts/clean_data.py or notebooks/Data_Cleaning.ipynb) that:

- Handles missing values (e.g., imputation, removal).
- Corrects or removes outliers.
- Ensures data consistency (e.g., uniform date formatting, unit conversions).
- Saves cleaned data in a new folder (data/clean/).

